In [4]:
import numpy as np
import pandas as pd

# Load data
try:
    measurements = pd.read_csv("measurements.csv").values
    groundtruth = pd.read_csv("groundtruth.csv").values
except Exception as e:
    print(f"Error reading files: {e}")
    exit()

# Validate data dimensions
assert measurements.shape[1] == 3, "Measurements must have 3 columns (3-axis accelerometer)."
assert groundtruth.shape[1] == 3, "Ground truth must have 3 columns (3-axis accelerometer)."

# Augment measurements for affine correction
n = measurements.shape[0]
R_augmented = np.hstack([measurements, np.ones((n, 1))])  # Add bias term

# Solve for A and b using least squares
params, _, _, _ = np.linalg.lstsq(R_augmented, groundtruth, rcond=None)
A = params[:3, :]  # 3x3 matrix
b = params[3, :]   # 3x1 vector

# Display results
print("Matrix A (Scale, Rotation, Shear):")
print(A)
print("Vector b (Offset):")
print(b)

# Calculate sum-of-squares error (SSE)
y_pred = measurements @ A + b
sse = np.sum(np.square(groundtruth - y_pred))
print(f"Sum-of-Squares Error (SSE): {sse}")

# Comments on correction function
print("\nInterpretation:")
print("- Offset (b): Corrects constant biases in measurements.")
print("- Scale (A): Adjusts the magnitude of measurements.")
print("- Rotation (A): Aligns measurement axes with ground truth.")
print("- Shear (A): Corrects inter-axis dependencies (non-orthogonality).")


Matrix A (Scale, Rotation, Shear):
[[ 9.99403482e-01 -3.48925650e-02 -4.14321784e-05]
 [ 3.49460657e-02  9.99394058e-01 -1.28268353e-05]
 [ 3.41963584e-05 -3.71087408e-05  8.33230905e-01]]
Vector b (Offset):
[-0.19160864  0.3040108  -0.00317657]
Sum-of-Squares Error (SSE): 0.07376304964818722

Interpretation:
- Offset (b): Corrects constant biases in measurements.
- Scale (A): Adjusts the magnitude of measurements.
- Rotation (A): Aligns measurement axes with ground truth.
- Shear (A): Corrects inter-axis dependencies (non-orthogonality).
